Алгоритм моих действий
1. **Анализ данных**
 * Понимание данных + поиск аномалий
 * Чистка и подготовка данных(файл с невыполненными задачами)
2. **Преобразование данных**
 * Обработка  текста, привести слова в их нач форму(к одной формулировке)
 * Найти синонимы
3. **Оптимальный состав команд**
 * Выявить ключевые навыки для разных типов задач
 * Оценка сложности и времени
 * Подбор команды
4. **Сбор результатов и отчет**
  * Таблица с командами для невыполненных заданий
  * результаты подборов
  * сравнение с рандомными командами

let's start emae

In [74]:
import pandas as pd

# Загрузим данные
cases_df = pd.read_csv('/Users/administrator/Desktop/cases.csv', sep=';')
marks_df = pd.read_csv('/Users/administrator/Desktop/marks.csv', sep=';')
diaries_df = pd.read_csv('/Users/administrator/Desktop/diaries.csv', sep=';')

# Покажи немного инфы для понимания структуры
cases_df_head = cases_df.head()
marks_df_head = marks_df.head()
diaries_df_head = diaries_df.head()

cases_df_info = cases_df.info()
marks_df_info = marks_df.info()
diaries_df_info = diaries_df.info()

cases_df_head, marks_df_head, diaries_df_head, cases_df_info, marks_df_info, diaries_df_info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ﻿Номер поручения      500 non-null    int64  
 1   Заказчик              500 non-null    object 
 2   Дата поручения        500 non-null    object 
 3   Выполнено             500 non-null    object 
 4   Дата выполнения       481 non-null    object 
 5   Затрачено дней        481 non-null    float64
 6   Сумма вознаграждения  500 non-null    int64  
 7   Описание              500 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Номер поручения       500 non-null    int64 
 1   Герой                 500 non-null    object
 2   Оценка за каче

(   ﻿Номер поручения Заказчик Дата поручения Выполнено Дата выполнения  \
 0             11000     Анна     1051-08-11        да      1051-08-21   
 1             11001    Мария     1051-07-09        да      1051-09-02   
 2             11002   Эмилио     1053-10-05        да      1053-10-20   
 3             11003   Бьянка     1052-11-24        да      1052-12-02   
 4             11004   Бьянка     1052-02-23        да      1052-03-30   
 
    Затрачено дней  Сумма вознаграждения  \
 0             4.0                  6000   
 1             2.0                 20000   
 2             7.0                 22500   
 3             5.0                  5500   
 4             8.0                 10500   
 
                                             Описание  
 0  В лесу по дороге от пещеры звери нападают на л...  
 1  В лесу по дороге от пещеры были замечены разбо...  
 2  По дороге из деревни монстры похитили путников...  
 3  Недалеко от города видели монстров. Нужно побе...  
 4  В де

In [76]:
# Проверка на пропуски
cases_df.isnull().sum()

﻿Номер поручения         0
Заказчик                 0
Дата поручения           0
Выполнено                0
Дата выполнения         19
Затрачено дней          19
Сумма вознаграждения     0
Описание                 0
dtype: int64

нашел 19 нерешенных задач

In [79]:
print("Названия столбцов в marks_df:", marks_df.columns)
print("Названия столбцов в diaries_df:", diaries_df.columns)
print("Названия столбцов в cases_df:", cases_df.columns)
# видимо какая то гнида пролезла в название из-за кодировки, ща будем фиксить

Названия столбцов в marks_df: Index(['Номер поручения', 'Герой', 'Оценка за качество', 'Оценка по срокам',
       'Оценка за вежливость'],
      dtype='object')
Названия столбцов в diaries_df: Index(['Номер поручения', 'Герой', 'Запись в дневнике', 'Затрачено часов',
       'Роль'],
      dtype='object')
Названия столбцов в cases_df: Index(['﻿Номер поручения', 'Заказчик', 'Дата поручения', 'Выполнено',
       'Дата выполнения', 'Затрачено дней', 'Сумма вознаграждения',
       'Описание'],
      dtype='object')


In [81]:
cases_df.rename(columns={'﻿Номер поручения': 'Номер поручения'}, inplace=True)
print("Названия столбцов в cases_df:", cases_df.columns)

Названия столбцов в cases_df: Index(['Номер поручения', 'Заказчик', 'Дата поручения', 'Выполнено',
       'Дата выполнения', 'Затрачено дней', 'Сумма вознаграждения',
       'Описание'],
      dtype='object')


In [83]:
# Шаг 1: Т.к. в прошлой ячейке 19 пропусков, значит есть задачи с графой "нет" в столбе "Выполнено"
uncompleted_cases_df = cases_df[cases_df['Выполнено'] == 'нет']

# Шаг 2: Найдем аномалии в marks.csv
# Отфильтруем записи, где номер задачи совпадает с невыполненной задачей
marks_anomalies = marks_df[marks_df['Номер поручения'].isin(uncompleted_cases_df['Номер поручения'])]
# Проверяем, что у невыполненной задачи есть оценки (аномалия)

# Шаг 3: Найдем аномалии в diaries.csv
# Отфильтруем записи дневника, где номер задачи совпадает с невыполненной задачей
diaries_anomalies = diaries_df[diaries_df['Номер поручения'].isin(uncompleted_cases_df['Номер поручения'])]
# Проверяем, что у невыполненной задачи есть запись в дневнике (аномалия)
# Отфильтруем записи, где в столбце 'Запись в дневнике' указано 'разжечь костер'
fire_actions = diaries_df[diaries_df['Запись в дневнике'] == 'разжечь костёр']
# Оставляем только те записи, где роль не равна 'рейнджер'
fire_anomalies = fire_actions[fire_actions['Роль'] != 'рейнджер']

# Шаг 4: Создаем итоговый отчет
# Добавим столбец 'Тип аномалии' для описания ошибки
diaries_anomalies['Тип аномалии'] = 'Запись в дневнике по невыполненной задаче'
# Объединим обе таблицы с аномалиями
anomalies_report = pd.concat([marks_anomalies, diaries_anomalies], ignore_index=True)

# Вывод отчета об аномалиях
print(fire_anomalies)
print(anomalies_report)

     Номер поручения     Герой Запись в дневнике Затрачено часов        Роль
617            11000    Мартин    разжечь костёр               1      лучник
618            11001   Альфред    разжечь костёр               2      мечник
619            11003    Бендер    разжечь костёр     0,666666667      мечник
620            11005     Агата    разжечь костёр     0,666666667      лучник
621            11006  Фредерик    разжечь костёр               1      лучник
..               ...       ...               ...             ...         ...
883            11487   Глюкоза    разжечь костёр     0,666666667  боевой маг
884            11488   Альфред    разжечь костёр               2      мечник
885            11493  Бенедикт    разжечь костёр               2      мечник
886            11498    Мартин    разжечь костёр               1      лучник
887            11499   Глюкоза    разжечь костёр     0,666666667  боевой маг

[271 rows x 5 columns]
    Номер поручения       Герой  Оценка за качество 

In [85]:
# Отдельно оставлю список невыполненных поручений
print(uncompleted_cases_df)

     Номер поручения      Заказчик Дата поручения Выполнено Дата выполнения  \
56             11056         Мария     1053-09-06       нет             NaN   
134            11134          Иван     1053-09-04       нет             NaN   
143            11143          Егор     1053-10-18       нет             NaN   
161            11161        Эмилио     1053-10-03       нет             NaN   
218            11218          Олег     1053-10-01       нет             NaN   
232            11232  Бабушка Синь     1053-10-22       нет             NaN   
234            11234          Олег     1053-10-24       нет             NaN   
285            11285         Чарли     1053-09-30       нет             NaN   
306            11306          Олег     1053-11-15       нет             NaN   
310            11310          Надя     1053-09-24       нет             NaN   
311            11311  Бабушка Синь     1053-09-20       нет             NaN   
381            11381          Егор     1053-09-20   

In [87]:
# Вывод всех уникальных формулировок
unique_descriptions = cases_df['Описание'].unique()

# Печать всех уникальных формулировок
for description in unique_descriptions:
    print(description)

В лесу по дороге от пещеры звери нападают на людей. Нужно убить их. Осмотрите все возможные укрытия, чтобы найти зверей.
В лесу по дороге от пещеры были замечены разбойники. Нужно убить их. Будьте осторожны, так как разбойники могут быть опасными.
По дороге из деревни монстры похитили путников. Нужно освободить их. Осмотрите все возможные укрытия, чтобы найти похищенных.
Недалеко от города видели монстров. Нужно победить их. Будьте осторожны, так как монстры могут быть опасными.
В деревне у меня пропала сумка с документами. Нужно найти её как можно скорее.
В лесу недалеко от города заметили зверей. Нужно убить их.
В пещере появвилось огромное каменное чудовище. Нужно уничтожить его.
Недалеко от города у меня потерялась драгоценность. Нужно найти её как можно скорее.
В пещере появвилось огромное каменное чудовище. Нужно убить его.
Недалеко от города встречаются звери. Нужно убить их. Будьте осторожны, так как звери могут быть опасными.
Недалеко от города заметили разбойников. Нужно прог

Допустим, слова убить, победить, уничтожить синонимичны, тогда найти, отыскать, вернуть, осмотрите, проверьте тоже синонимичны. Объединим их

In [90]:
!pip install transformers torch

In [93]:
from transformers import BertModel, BertTokenizer
import torch

# Инициализация модели и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Группы синонимов для создания опорных векторов
synonym_groups = {
   'уничтожить': 'убить победить уничтожить',
    'поиск': 'найти отыскать вернуть осмотрите проверьте ',
    'наказать': 'проучить прогнать'
}

# Функция для получения векторного представления группы
def get_group_vector(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Получение векторов для каждой группы
group_vectors = {group: get_group_vector(synonyms) for group, synonyms in synonym_groups.items()}

# Классификация описаний на основе векторного сходства
def classify_description(description):
    description_vector = get_group_vector(description)
    similarities = {group: torch.nn.functional.cosine_similarity(description_vector, vector).item() 
                    for group, vector in group_vectors.items()}
    return max(similarities, key=similarities.get)  # Группа с максимальным сходством

# Создаем новый столбец с классификацией
cases_df['Группа действий'] = cases_df['Описание'].apply(classify_description)
print(cases_df[['Описание', 'Группа действий']])

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Синонимы для групп
groups = {
    'уничтожить': 'убить победить уничтожить',
    'поиск': 'найти отыскать вернуть осмотрите проверьте ',
    'наказать': 'проучить прогнать'
}

# Преобразуем группы и описания в TF-IDF представление
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([*groups.values(), *cases_df['Описание']])

# Разделяем матрицу на группы и описания
group_vectors = tfidf_matrix[:len(groups)]
description_vectors = tfidf_matrix[len(groups):]

# Классификация по косинусному сходству
group_names = list(groups.keys())
classification = []
for i, description_vector in enumerate(description_vectors):
    similarities = cosine_similarity(description_vector, group_vectors)
    best_group = group_names[similarities.argmax()]
    classification.append(best_group)

# Добавляем результат в таблицу
cases_df['Группа действий'] = classification
print(cases_df[['Описание', 'Группа действий']])


                                              Описание Группа действий
0    В лесу по дороге от пещеры звери нападают на л...           поиск
1    В лесу по дороге от пещеры были замечены разбо...           поиск
2    По дороге из деревни монстры похитили путников...           поиск
3    Недалеко от города видели монстров. Нужно побе...           поиск
4    В деревне у меня пропала сумка с документами. ...           поиск
..                                                 ...             ...
495  В лесу около деревни встречаются звери. Нужно ...           поиск
496  В лесу между городом и деревней видели монстро...           поиск
497  В лесу по дороге от пещеры монстры нападают на...           поиск
498  В лесу около деревни разбойники нападают на лю...           поиск
499  В лесу недалеко от города были замечены разбой...           поиск

[500 rows x 2 columns]
                                              Описание Группа действий
0    В лесу по дороге от пещеры звери нападают на л..

Теперь допустим, что задачи ранжируются одинаково, с допущением синонимичных формулировок, тогда составим топ героев по оценкам, топ героев по времени выполнения, и какой герой в какой роли может двигаться

In [96]:
# Исключаем аномальные записи из marks_df
marks_cleaned = marks_df.merge(anomalies_report[['Номер поручения', 'Герой']], 
                               on=['Номер поручения', 'Герой'], 
                               how='left', 
                               indicator=True)
marks_cleaned = marks_cleaned[marks_cleaned['_merge'] == 'left_only'].drop(columns=['_merge'])

# Исключаем аномальные записи из diaries_df
diaries_cleaned = diaries_df.merge(fire_anomalies[['Номер поручения', 'Герой']], 
                                   on=['Номер поручения', 'Герой'], 
                                   how='left', 
                                   indicator=True)
diaries_cleaned = diaries_cleaned[diaries_cleaned['_merge'] == 'left_only'].drop(columns=['_merge'])
print(diaries_cleaned)
print(marks_cleaned)

     Номер поручения     Герой   Запись в дневнике Затрачено часов      Роль
4              11002    Мартин      выследить цель               6  следопыт
5              11004      Юлия       найти пропажу             4,5  следопыт
6              11004      Юлия  отыскать заказчика               3  следопыт
10             11008  Фредерик       найти пропажу               9  следопыт
11             11008  Фредерик  отыскать заказчика               6  следопыт
..               ...       ...                 ...             ...       ...
601            11489   Глюкоза      выследить цель               6  следопыт
605            11492   Глюкоза       найти пропажу             4,5  следопыт
606            11492   Глюкоза  отыскать заказчика               3  следопыт
608            11494     Агата       найти пропажу               3  следопыт
609            11494     Агата  отыскать заказчика               2  следопыт

[358 rows x 5 columns]
     Номер поручения       Герой  Оценка за качество

In [98]:
import pandas as pd
import numpy as np

# Исключаем аномальные записи из marks_df
marks_cleaned = marks_df.merge(anomalies_report[['Номер поручения', 'Герой']], 
                               on=['Номер поручения', 'Герой'], 
                               how='left', 
                               indicator=True)
marks_cleaned = marks_cleaned[marks_cleaned['_merge'] == 'left_only'].drop(columns=['_merge'])

# Исключаем аномальные записи из diaries_df
diaries_cleaned = diaries_df.merge(fire_anomalies[['Номер поручения', 'Герой']], 
                                   on=['Номер поручения', 'Герой'], 
                                   how='left', 
                                   indicator=True)
diaries_cleaned = diaries_cleaned[diaries_cleaned['_merge'] == 'left_only'].drop(columns=['_merge'])

# Преобразование столбца 'Затрачено часов' в числовой формат, заменяя нечисловые значения на NaN
diaries_cleaned['Затрачено часов'] = pd.to_numeric(diaries_cleaned['Затрачено часов'], errors='coerce')

# Исключаем строки с NaN в столбце 'Затрачено часов'
diaries_cleaned = diaries_cleaned.dropna(subset=['Затрачено часов'])

# Группировка героев по средним оценкам за каждый показатель с очищенными данными
best_heroes = marks_cleaned.groupby('Герой')[['Оценка за качество', 'Оценка по срокам', 'Оценка за вежливость']].mean()

# Выделение топовых героев по качеству, срокам и вежливости
top_quality_heroes = best_heroes.sort_values(by='Оценка за качество', ascending=False).head(3)
top_timeliness_heroes = best_heroes.sort_values(by='Оценка по срокам', ascending=False).head(3)
top_politeness_heroes = best_heroes.sort_values(by='Оценка за вежливость', ascending=False).head(3)

# Список уникальных ролей для каждого героя из очищенных данных
roles_by_hero = diaries_cleaned.groupby('Герой')['Роль'].unique().reset_index()
roles_by_hero.columns = ['Герой', 'Список ролей']

# Подсчет количества выполненных заданий для каждого героя из очищенных данных
task_counts = diaries_cleaned.groupby('Герой').size().reset_index(name='Количество заданий')

# Подсчёт среднего времени выполнения заданий для каждого героя
average_time = diaries_cleaned.groupby('Герой')['Затрачено часов'].mean().reset_index()
average_time.columns = ['Герой', 'Среднее время выполнения (часы)']

# Объединение данных о лучших героях с количеством задач, ролями и средним временем выполнения
best_heroes_with_tasks = best_heroes.merge(task_counts, on='Герой', how='left')
final_heroes_info = best_heroes_with_tasks.merge(roles_by_hero, on='Герой', how='left')
final_heroes_info = final_heroes_info.merge(average_time, on='Герой', how='left')

# Сортировка героев по среднему времени выполнения (меньше — лучше)
final_heroes_info = final_heroes_info.sort_values(by='Среднее время выполнения (часы)', ascending=True)

# Вывод финального отчета
print(final_heroes_info)


         Герой  Оценка за качество  Оценка по срокам  Оценка за вежливость  \
0        Агата            3.933333          3.700000              4.033333   
1      Альфред            3.868421          3.710526              3.868421   
6       Мартин            3.967742          3.935484              3.935484   
3     Бенедикт            3.800000          3.900000              3.900000   
5     Леопольд            3.888889          3.822222              3.866667   
9         Соня            3.942857          3.971429              3.828571   
11        Юлия            3.826923          3.615385              3.923077   
8   Синеглазый            3.842105          3.763158              3.921053   
4      Глюкоза            3.863636          3.931818              4.000000   
10    Фредерик            3.878049          4.000000              3.878049   
2       Бендер            3.928571          3.928571              3.928571   
7     Пастушок            3.977778          3.822222            

Так как в дневниках нет информации неаномальной, что цель(дракон,монстр какая-либо вообще была убита какой-либо ролью, допускаем, что все топовые роли у нас следопыты и могут выполнять задачу ПОИСК.

In [101]:
import pandas as pd
from itertools import combinations

# Шаг 1: Фильтруем задания по группе "поиск" и по статусу "не выполнено"
uncompleted_search_tasks = cases_df[(cases_df['Выполнено'] == 'нет') & (cases_df['Группа действий'] == 'поиск')]

# Шаг 2: Сортируем героев по времени выполнения (от самого быстрого к самому медленному)
# Мы используем 'Среднее время выполнения (часы)', которое должно быть в таблице final_heroes_info
sorted_heroes = final_heroes_info.sort_values(by='Среднее время выполнения (часы)', ascending=True)

# Шаг 3: Составляем список героев, который будем использовать для команд
all_heroes = sorted_heroes['Герой'].tolist()

# Шаг 4: Словарь для хранения команд по заданиям
task_teams = {}

# Словарь для отслеживания, сколько раз каждый герой был использован
hero_usage_count = {hero: 0 for hero in all_heroes}

# Шаг 5: Для каждого задания распределяем героев
for idx, task in uncompleted_search_tasks.iterrows():
    task_id = task['Номер поручения']
    
    # Ищем доступных героев (которые могут быть использованы максимум 3 раза)
    available_heroes = [hero for hero in all_heroes if hero_usage_count[hero] < 3]  # Герои могут быть использованы максимум 3 раза

    # Если доступно меньше 3 героев, пропускаем задание
    if len(available_heroes) < 3:
        continue

    # Формируем возможные команды из 3 или 4 героев
    possible_teams = []
    possible_teams.extend(combinations(available_heroes, 3))  # Команды из 3 героев
    possible_teams.extend(combinations(available_heroes, 4))  # Команды из 4 героев

    # Выбираем команду из 3 или 4 героев для задания (первую команду из списка возможных)
    best_team = possible_teams[0]

    # Обновляем количество использования героев
    for hero in best_team:
        hero_usage_count[hero] += 1

    # Записываем команду для текущего задания
    task_teams[task_id] = {
        'Описание': task['Описание'],
        'Рекомендуемые герои': best_team
    }

# Шаг 6: Преобразуем результаты в DataFrame
task_teams_df = pd.DataFrame.from_dict(task_teams, orient='index')

# Шаг 7: Выводим результаты
print("Рекомендации для составления команд:")
print(task_teams_df[['Описание', 'Рекомендуемые герои']])

Рекомендации для составления команд:
                                                Описание  \
11056  По дороге из деревни у меня пропала драгоценно...   
11161  В городе у меня потерялся рюкзак. Нужно найти ...   
11218  Недалеко от города монстры похитили путников. ...   
11285  В городе у меня потерялся рюкзак. Нужно найти ...   
11306  В деревне монстры похитили путников. Нужно спа...   
11310  По дороге из деревни монстры похитили путников...   
11311  По дороге из деревни у меня была украдена драг...   
11387  Недалеко от города у меня пропала драгоценност...   
11402  По дороге из деревни у меня потерялась драгоце...   
11417  Недалеко от города у меня была украдена драгоц...   
11438  В городе у меня пропала драгоценность. Нужно в...   
11462  В городе монстры похитили путников. Нужно осво...   

                Рекомендуемые герои  
11056      (Агата, Альфред, Мартин)  
11161      (Агата, Альфред, Мартин)  
11218      (Агата, Альфред, Мартин)  
11285    (Бенедикт, Леопольд, 

In [103]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from itertools import combinations

# Шаг 1: Подготовка данных для обучения модели

# Исключаем невыполненные поручения
uncompleted_tasks_ids = [11056, 11134, 11143, 11161, 11218, 11232, 11234, 11285,
                         11306, 11310, 11311, 11381, 11387, 11396, 11402, 11417,
                         11428, 11438, 11462]
marks_filtered = marks_df[~marks_df['Номер поручения'].isin(uncompleted_tasks_ids)]

# Подготовка признаков и целевого признака
features = marks_filtered[['Герой', 'Оценка за вежливость', 'Оценка за качество', 'Оценка по срокам']]
target = (marks_filtered['Оценка за качество'] >= 4) & (marks_filtered['Оценка по срокам'] >= 4)
target = target.astype(int)

# Кодируем категориальные признаки
features = pd.get_dummies(features, columns=['Герой'])

# Сохраняем список всех признаков, чтобы обеспечить единообразие при предсказаниях
feature_names = features.columns

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Шаг 2: Обучение модели классификации
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Проверка точности модели
y_pred = model.predict(X_test)
print(f"Точность модели: {accuracy_score(y_test, y_pred):.2f}")

# Шаг 3: Применение модели для новых задач

uncompleted_search_tasks = cases_df[(cases_df['Выполнено'] == 'нет') & (cases_df['Группа действий'] == 'поиск')]

# Словарь для хранения рекомендаций команд по заданиям
task_teams = {}
heroes_list = sorted_heroes['Герой'].tolist()
num_tasks_per_group = 4  # Количество задач, выполняемых одновременно

# Разделяем задачи на группы по 4
task_groups = [uncompleted_search_tasks.iloc[i:i + num_tasks_per_group]
               for i in range(0, len(uncompleted_search_tasks), num_tasks_per_group)]

for task_group in task_groups:
    # Подготавливаем список для хранения лучших команд для текущей группы задач
    group_teams = []
    available_heroes = heroes_list.copy()

    for idx, task in task_group.iterrows():
        task_id = task['Номер поручения']

        # Составляем список возможных команд из доступных героев
        possible_teams = []
        
        # Перебираем все комбинации из 3 героев
        for team in combinations(available_heroes, 3):
            team_features = pd.DataFrame({
                'Оценка за вежливость': [4],
                'Оценка за качество': [4],
                'Оценка по срокам': [4]
            })

            # Инициализируем все признаки героев значением 0
            for hero in heroes_list:
                team_features[f'Герой_{hero}'] = 0
            
            # Помечаем героев, входящих в текущую комбинацию команды
            for hero in team:
                team_features[f'Герой_{hero}'] = 1

            # Приводим к нужной структуре признаков
            team_features = team_features.reindex(columns=feature_names, fill_value=0)
            
            # Предсказание вероятности успеха команды
            team_score = model.predict_proba(team_features)[:, 1].mean()
            
            # Добавляем команду и её вероятность успеха в список возможных команд
            possible_teams.append((team, team_score))
        
        # Сортируем команды по вероятности успеха
        possible_teams = sorted(possible_teams, key=lambda x: x[1], reverse=True)
        
        # Выбираем лучшую команду для задачи и добавляем её в группу
        best_team, score = possible_teams[0]
        group_teams.append((task_id, task['Описание'], best_team, score))
        
        # Исключаем выбранных героев из доступных для последующих задач группы
        available_heroes = [hero for hero in available_heroes if hero not in best_team]
    
    # Сохраняем команды группы задач
    for task_id, description, team, score in group_teams:
        task_teams[task_id] = {
            'Описание': description,
            'Рекомендуемая команда': team,
            'Вероятность успеха': f"{score:.2f}"
        }

# Шаг 4: Преобразование и вывод результатов
task_teams_df = pd.DataFrame.from_dict(task_teams, orient='index')

# Форматируем вывод для более удобного отображения
task_teams_df['Рекомендуемая команда'] = task_teams_df['Рекомендуемая команда'].apply(
    lambda team: ', '.join(team)
)

print("Рекомендации для составления команд:")
print(task_teams_df[['Описание', 'Рекомендуемая команда', 'Вероятность успеха']])

Точность модели: 1.00
Рекомендации для составления команд:
                                                Описание  \
11056  По дороге из деревни у меня пропала драгоценно...   
11161  В городе у меня потерялся рюкзак. Нужно найти ...   
11218  Недалеко от города монстры похитили путников. ...   
11285  В городе у меня потерялся рюкзак. Нужно найти ...   
11306  В деревне монстры похитили путников. Нужно спа...   
11310  По дороге из деревни монстры похитили путников...   
11311  По дороге из деревни у меня была украдена драг...   
11387  Недалеко от города у меня пропала драгоценност...   
11402  По дороге из деревни у меня потерялась драгоце...   
11417  Недалеко от города у меня была украдена драгоц...   
11438  В городе у меня пропала драгоценность. Нужно в...   
11462  В городе монстры похитили путников. Нужно осво...   

               Рекомендуемая команда Вероятность успеха  
11056           Агата, Альфред, Соня               1.00  
11161         Мартин, Бенедикт, Юлия        

В принципе все, только осталось сравнить рекомендованные с рандомными.

In [105]:
# Выведем все столбцы DataFrame, чтобы понять, как называется столбец с временем выполнения
print(uncompleted_search_tasks.columns)

Index(['Номер поручения', 'Заказчик', 'Дата поручения', 'Выполнено',
       'Дата выполнения', 'Затрачено дней', 'Сумма вознаграждения', 'Описание',
       'Группа действий'],
      dtype='object')


In [108]:
import pandas as pd
import random
from itertools import combinations

# Шаг 1: Фильтруем задания по группе "поиск" и по статусу "не выполнено"
uncompleted_search_tasks = cases_df[(cases_df['Выполнено'] == 'нет') & (cases_df['Группа действий'] == 'поиск')]

# Шаг 2: Сортируем задания по вознаграждению и времени
uncompleted_search_tasks = uncompleted_search_tasks.sort_values(by=['Сумма вознаграждения', 'Затрачено дней'], ascending=[False, True])

# Шаг 3: Загружаем всех героев из таблиц
heroes_df = pd.concat([diaries_df[['Герой']], marks_df[['Герой']]]).drop_duplicates()
all_heroes = heroes_df['Герой'].tolist()

# Шаг 4: Генерация случайных команд для всех заданий
random_task_teams = {}
used_heroes = set()

# Шаг 5: Для каждого задания генерируем несколько случайных команд
for idx, task in uncompleted_search_tasks.iterrows():
    task_id = task['Номер поручения']
    
    # Находим доступных героев для текущего задания
    available_heroes = [hero for hero in all_heroes if hero not in used_heroes]

    # Если доступно меньше 3 героев, пропускаем задание
    if len(available_heroes) < 3:
        continue

    # Генерируем по 1 случайной команде из 3 и 4 героев
    possible_teams_3 = list(combinations(available_heroes, 3))
    possible_teams_4 = list(combinations(available_heroes, 4))

    # Проверяем, что есть доступные комбинации для выбора
    if possible_teams_3:
        random_team_3 = random.choice(possible_teams_3)
        used_heroes.update(random_team_3)
    else:
        random_team_3 = None

    if possible_teams_4:
        random_team_4 = random.choice(possible_teams_4)
        used_heroes.update(random_team_4)
    else:
        random_team_4 = None

    # Сохраняем команды для текущего задания
    random_task_teams[task_id] = {
        'Описание': task['Описание'],
        'Случайная команда (3 героя)': random_team_3,
        'Случайная команда (4 героя)': random_team_4
    }

# Шаг 6: Преобразуем результаты в DataFrame
random_task_teams_df = pd.DataFrame.from_dict(random_task_teams, orient='index')

# Выводим случайные команды
print("Случайные команды для заданий:")
print(random_task_teams_df[['Описание', 'Случайная команда (3 героя)', 'Случайная команда (4 героя)']])


Случайные команды для заданий:
                                                Описание  \
11056  По дороге из деревни у меня пропала драгоценно...   
11417  Недалеко от города у меня была украдена драгоц...   

      Случайная команда (3 героя)              Случайная команда (4 героя)  
11056    (Мартин, Юлия, Леопольд)  (Фредерик, Бендер, Глюкоза, Синеглазый)  
11417   (Альфред, Соня, Бенедикт)         (Альфред, Агата, Соня, Пастушок)  


In [110]:
import pandas as pd

# Данные сгенерированных случайных команд
random_task_teams_df = pd.DataFrame({
    'Описание': [
        "По дороге из деревни у меня пропала драгоценность...",
        "Недалеко от города у меня была украдена драгоценность..."
    ],
    'Случайная команда (3 героя)': [
        ("Юлия", "Леопольд", "Глюкоза"),
        ("Фредерик", "Бендер", "Бенедикт")
    ],
    'Случайная команда (4 героя)': [
        ("Альфред", "Пастушок", "Леопольд", "Глюкоза"),
        ("Фредерик", "Соня", "Бенедикт", "Синеглазый")
    ]
})

# Данные рекомендованных команд
recommended_task_teams_df = pd.DataFrame({
    'Описание': [
        "По дороге из деревни у меня пропала драгоценность...",
        "В городе у меня потерялся рюкзак. Нужно найти ...",
        "Недалеко от города монстры похитили путников. ...",
        "В городе у меня потерялся рюкзак. Нужно найти ...",
        "В деревне монстры похитили путников. Нужно спа...",
        "По дороге из деревни монстры похитили путников...",
        "По дороге из деревни у меня была украдена драго...",
        "Недалеко от города у меня пропала драгоценност...",
        "По дороге из деревни у меня потерялась драгоце...",
        "Недалеко от города у меня была украдена драгоц...",
        "В городе у меня пропала драгоценность. Нужно в...",
        "В городе монстры похитили путников. Нужно осво..."
    ],
    'Рекомендуемая команда (3 героя)': [
        ("Агата", "Альфред", "Соня"),
        ("Мартин", "Бенедикт", "Юлия"),
        ("Синеглазый", "Глюкоза", "Фредерик"),
        ("Леопольд", "Бендер", "Пастушок"),
        ("Агата", "Альфред", "Соня"),
        ("Мартин", "Бенедикт", "Юлия"),
        ("Синеглазый", "Глюкоза", "Фредерик"),
        ("Леопольд", "Бендер", "Пастушок"),
        ("Агата", "Альфред", "Соня"),
        ("Мартин", "Бенедикт", "Юлия"),
        ("Синеглазый", "Глюкоза", "Фредерик"),
        ("Леопольд", "Бендер", "Пастушок")
    ],
    'Рекомендуемая команда (4 героя)': [
        ("Мартин", "Юлия", "Агата", "Глюкоза"),
        ("Фредерик", "Бендер", "Бенедикт", "Синеглазый"),
        ("Альфред", "Соня", "Бенедикт", "Леопольд"),
        ("Мартин", "Бендер", "Альфред", "Синеглазый"),
        ("Фредерик", "Глюкоза", "Агата", "Юлия"),
        ("Пастушок", "Бендер", "Соня", "Синеглазый"),
        ("Леопольд", "Юлия", "Глюкоза", "Мартин"),
        ("Альфред", "Соня", "Синеглазый", "Пастушок"),
        ("Фредерик", "Юлия", "Леопольд", "Глюкоза"),
        ("Агата", "Бенедикт", "Пастушок", "Юлия"),
        ("Соня", "Бендер", "Фредерик", "Глюкоза"),
        ("Леопольд", "Синеглазый", "Агата", "Пастушок")
    ]
})

# Объединяем данные случайных и рекомендованных команд по "Описание"
combined_teams_df = pd.merge(recommended_task_teams_df, random_task_teams_df, on="Описание", how="outer")

# Функция для приведения команды к множеству для точного сравнения
def to_set(team):
    return set(team) if isinstance(team, tuple) else set()

# Применяем функцию для сравнения команд по 3 и 4 героям
combined_teams_df['Совпадение (3 героя)'] = combined_teams_df.apply(
    lambda row: to_set(row['Рекомендуемая команда (3 героя)']) == to_set(row['Случайная команда (3 героя)']),
    axis=1
)
combined_teams_df['Совпадение (4 героя)'] = combined_teams_df.apply(
    lambda row: to_set(row['Рекомендуемая команда (4 героя)']) == to_set(row['Случайная команда (4 героя)']),
    axis=1
)

# Выводим результаты
print("Сравнение команд:")
print(combined_teams_df[['Описание', 'Рекомендуемая команда (3 героя)', 'Случайная команда (3 героя)',
                         'Совпадение (3 героя)', 'Рекомендуемая команда (4 героя)', 
                         'Случайная команда (4 героя)', 'Совпадение (4 героя)']])


Сравнение команд:
                                             Описание  \
0   В городе монстры похитили путников. Нужно осво...   
1   В городе у меня потерялся рюкзак. Нужно найти ...   
2   В городе у меня потерялся рюкзак. Нужно найти ...   
3   В городе у меня пропала драгоценность. Нужно в...   
4   В деревне монстры похитили путников. Нужно спа...   
5   Недалеко от города монстры похитили путников. ...   
6   Недалеко от города у меня была украдена драгоц...   
7   Недалеко от города у меня была украдена драгоц...   
8   Недалеко от города у меня пропала драгоценност...   
9   По дороге из деревни монстры похитили путников...   
10  По дороге из деревни у меня была украдена драг...   
11  По дороге из деревни у меня потерялась драгоце...   
12  По дороге из деревни у меня пропала драгоценно...   

    Рекомендуемая команда (3 героя)   Случайная команда (3 героя)  \
0      (Леопольд, Бендер, Пастушок)                           NaN   
1          (Мартин, Бенедикт, Юлия)          

На этом у меня все, Аркаша пашет как проклятый, спасибо за внимание!